# 8. Ensemeble (앙상블 방법)

### 결정 트리의 약점인 과적합으로 인한 예측력을 보완

### Bagging (배깅)
> 훈련데이터에서 랜덤하게 추출된 샘플데이터를 통해 약한 의사결정나무모형을 만들어 각 트리모형의 예측치의 평균(회귀모형인 경우) 또는 다수결의 방법으로 결합하는 앙상블 방법

In [219]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [312]:
import matplotlib.pyplot as plt
from sklearn.ensemble import BaggingClassifier as bagging
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier as DT

In [233]:
cancer = load_breast_cancer()

In [256]:
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names)
df["class"] = cancer.target
df['class'] = df['class'].map({0: 'negative', 1:'positive'})
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,negative
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,negative
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,negative
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,negative
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,negative
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,negative
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,negative
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,negative


### 데이터 분할

In [278]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### 트리 모형 생성

In [300]:
tree = DT(criterion='entropy', min_samples_split=4)
tree.fit(X_train, y_train)
bag_trees =  bagging(DT(), n_estimators=1000, max_samples=0.5, random_state=0)
# 변수 : (배깅트리(기본모형을 DT로 지정), 기본모형의 수, 랜덤샘플링 비율, 랜덤 스테이트)
bag_trees.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=1000, random_state=0)

In [301]:
# 학습된 모델에 테스트 데이터 적용하여 예측 결과
y_pred_tree = tree.predict(X_test)
y_pred_bag = bag_trees.predict(X_test)
y_pred_tree, y_pred_bag

(array(['negative', 'positive', 'positive', 'positive', 'positive',
        'positive', 'positive', 'positive', 'positive', 'positive',
        'positive', 'positive', 'positive', 'negative', 'positive',
        'negative', 'positive', 'negative', 'negative', 'negative',
        'negative', 'negative', 'positive', 'positive', 'negative',
        'positive', 'positive', 'negative', 'positive', 'negative',
        'positive', 'negative', 'positive', 'negative', 'positive',
        'negative', 'positive', 'negative', 'positive', 'negative',
        'negative', 'positive', 'negative', 'positive', 'positive',
        'negative', 'positive', 'negative', 'positive', 'negative',
        'negative', 'negative', 'negative', 'positive', 'positive',
        'positive', 'positive', 'positive', 'positive', 'negative',
        'negative', 'negative', 'positive', 'positive', 'negative',
        'positive', 'negative', 'negative', 'negative', 'positive',
        'negative', 'negative', 'positive', 'pos

In [311]:
# 테스트 데이터 적용 시 예측력 평가 (배깅시, 일반 트리 순서)
print (bag_trees.score(X_test, y_test), tree.score(X_test, y_test))

# 예측 데이터와 테스트 데이터 차이의 점수 평가 (배깅시, 일반 트리 순서)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred_bag), accuracy_score(y_test, y_pred_tree))

0.9736842105263158 0.9385964912280702
0.9736842105263158 0.9385964912280702


### 결론 : 전체적으로 배깅이 일반적인 결정 트리 모델보다 정확성이 뛰어난 것을 확인 가능

In [303]:
# 예측 데이터 예측 유무 클래스 확인
tree.predict_proba(X_test), bag_trees.predict_proba(X_test)

(array([[1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0

### (참고)결정트리 시각화

In [309]:
# from sklearn.tree import export_graphviz
# #시각화 결과를 tree.dot 파일로 저장
# export_graphviz(tree, out_file='tree.dot', class_names=cancer.target_names, feature_names=cancer.feature_names, impurity=True, filled=True)

In [310]:
# import graphviz

# with open("tree.dot") as f:
#     dot_graph = f.read()

# graphviz.Source(dot_graph)

### Random Forest (랜덤 포레스트)
> 배깅방법에 비해 기본모형 생성시 더 많은 무작위성을 부여하는 방법 <br> 예측력이 배깅보다 더 높고, 부스팅에 비해 과적합이 일어나지 않는다는 장점이 있음

### Boosting (부스팅)
> 예측력이 약한 모형(weak learner)들을 결합하여 강한 예측모형을 만들기 위해 고안됨<br>약한 예측모형이란 랜덤하게 예측하는 것보다 약간 좋은 예측력을 지닌 모형